# DATA PROCESSING AND CALCULATION OF xG-ADJUSTED FPL POINTS

## Setup and pre-processing

In [1]:
# give the number of the latest FPL round
latest_gameweek = 31

In [2]:
# import basic libraries
import pandas as pd
import numpy as np
import json
import requests
from scipy.stats import poisson

# allow more data columns to be shown than by default
pd.set_option('max_columns',100)

In [3]:
# import player data 
filepath = '../data/fbref/player_stats_week' + str(latest_gameweek) + '.csv'
playerStats = pd.read_csv(filepath, index_col=0, skiprows=1)
playerStats.head()

,Player,Nation,Pos,Squad,Age,Born,MP,Starts,Min,Gls,Ast,PK,PKatt,CrdY,CrdR,Gls.1,Ast.1,G+A,G-PK,G+A-PK,xG,npxG,xA,xG.1,xA.1,xG+xA,npxG.1,npxG+xA,Matches
Rk,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,Patrick van Aanholt\Patrick-van-Aanholt,nl NED,DF,Crystal Palace,28,1990,24,24,2072,3,1,1,1,0,0,0.13,0.04,0.17,0.09,0.13,1.8,1.0,2.1,0.08,0.09,0.17,0.04,0.14,Matches
2,Max Aarons\Max-Aarons,eng ENG,DF,Norwich City,19,2000,29,29,2610,0,1,0,0,7,0,0.00,0.03,0.03,0.00,0.03,0.5,0.5,3.5,0.02,0.12,0.14,0.02,0.14,Matches
3,Tammy Abraham\Tammy-Abraham,eng ENG,FW,Chelsea,21,1997,27,23,1972,13,3,0,0,2,0,0.59,0.14,0.73,0.59,0.73,13.2,13.2,2.3,0.60,0.11,0.71,0.60,0.71,Matches
4,Che Adams\Che-Adams,eng ENG,FW,Southampton,23,1996,23,8,740,0,2,0,0,0,0,0.00,0.24,0.24,0.00,0.24,1.9,1.9,1.0,0.23,0.12,0.35,0.23,0.35,Matches
5,Adrián\Adrian,es ESP,GK,Liverpool,32,1987,11,9,875,0,0,0,0,1,0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,Matches


In [4]:
# import team data and pre-process
filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats = pd.read_csv(filepath, index_col=0)#, engine='python')

# change team names to match convention used in the FPL data
teamStats.loc[teamStats['Squad']=='Brighton & Hove Albion','Squad'] = 'Brighton'
teamStats.loc[teamStats['Squad']=='Manchester United','Squad'] = 'Manchester Utd'
teamStats.loc[teamStats['Squad']=='Newcastle United','Squad'] = 'Newcastle Utd'
teamStats.loc[teamStats['Squad']=='Sheffield United','Squad'] = 'Sheffield Utd'
teamStats.loc[teamStats['Squad']=='West Ham United','Squad'] = 'West Ham'
teamStats.loc[teamStats['Squad']=='Tottenham Hotspur','Squad'] = 'Tottenham'
teamStats.loc[teamStats['Squad']=='Wolverhampton Wanderers','Squad'] = 'Wolves'

teamStats.head()

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes
Rk,,,,,,,,,,,,,,,,,,
1,Liverpool,31,28,2,1,70,21,49,86,59.7,30.9,28.8,0.93,W L W D W,53143,Mohamed Salah - 17,Alisson,NaN
2,Manchester City,31,20,3,8,77,33,44,63,72.2,29.4,42.8,1.38,W L W W L,54219,Sergio Agüero - 16,Ederson,NaN
3,Leicester City,31,16,7,8,59,29,30,55,48.6,37.4,11.2,0.36,L L W D D,32061,Jamie Vardy - 19,Kasper Schmeichel,NaN
4,Chelsea,31,16,6,9,55,41,14,54,54.7,30.0,24.7,0.80,W D W W W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN
5,Manchester Utd,31,13,10,8,48,31,17,49,48.3,29.9,18.4,0.59,W D W D W,72726,Marcus Rashford Anthony Martial - 14,David de Gea,NaN


In [5]:
# fetch FPL data online
data = json.loads(requests.get('https://fantasy.premierleague.com/api/bootstrap-static/').text)
df = pd.DataFrame(data['elements'])
df.set_index('id',inplace=True)

# fetch data locally
#df = pd.read_csv('../data/data_week' + str(latest_gameweek) + '.csv', index_col=0)#,encoding='latin-1')

df.head()

,assists,bonus,bps,chance_of_playing_next_round,chance_of_playing_this_round,clean_sheets,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,creativity,creativity_rank,creativity_rank_type,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,goals_conceded,goals_scored,ict_index,ict_index_rank,ict_index_rank_type,in_dreamteam,influence,influence_rank,influence_rank_type,minutes,news,news_added,now_cost,own_goals,penalties_missed,penalties_saved,photo,points_per_game,red_cards,saves,second_name,selected_by_percent,special,squad_number,status,team,team_code,threat,threat_rank,threat_rank_type,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,yellow_cards
id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2,1,189,100.0,100.0,3,69140,0,0,-4,4,41.5,331,112,0,2,3.3,2.8,5,Shkodran,2.3,14,0,38.3,315,113,False,215.8,264,105,890,,2020-02-27T23:00:18.104137Z,51,0,0,0,69140.jpg,3.0,0,0,Mustafi,0.4,False,None,a,1,3,124.0,235,61,33,20707,1113,42031,367,0.5,6.5,Mustafi,1
2,0,3,153,100.0,100.0,3,98745,-1,1,-1,1,42.5,330,111,0,2,3.7,3.2,6,Héctor,2.7,15,1,25.8,354,128,False,155.4,313,123,893,,2019-12-09T20:00:21.228098Z,54,0,0,0,98745.jpg,3.0,0,0,Bellerín,1.2,False,None,a,1,3,58.0,314,107,33,191041,4676,126891,3931,0.5,6.1,Bellerín,2
3,2,1,243,100.0,100.0,2,111457,0,0,-3,3,159.8,198,49,0,2,1.7,1.2,1,Sead,0.7,21,0,43.1,295,102,False,206.2,272,109,1213,,2020-02-23T18:30:13.672943Z,52,0,0,0,111457.jpg,1.9,0,0,Kolasinac,0.5,False,None,a,1,3,67.0,303,99,37,60838,330,131395,548,0.1,7.1,Kolasinac,3
4,2,3,222,100.0,100.0,3,154043,0,0,-5,5,158.3,202,51,1,2,1.7,1.2,1,Ainsley,0.7,20,0,45.9,282,95,False,269.4,228,92,1243,,2019-09-22T18:00:10.824841Z,45,0,0,0,154043.jpg,2.4,1,0,Maitland-Niles,2.3,False,None,a,1,3,37.0,344,123,38,606662,565,644497,1664,0.2,8.4,Maitland-Niles,3
5,0,5,305,100.0,0.0,4,39476,0,0,-2,2,36.8,339,115,1,2,1.0,0.0,0,Sokratis,0.0,25,2,58.5,229,68,False,436.2,124,49,1696,,2020-06-18T18:00:15.974146Z,48,0,0,0,39476.jpg,3.0,0,0,Papastathopoulos,1.4,False,None,a,1,3,110.0,249,68,57,180102,517,221563,2686,0.0,11.9,Sokratis,6


## Probability to keep a clean sheet

Here, we estimate for each team the probability that the team keeps a clean sheet (against average opposition). We do this by first calculating the expected goals allowed per game for each team. Then, we assume that conceding goals follows a Poisson distribution, from which we then get the desired probability.

In [6]:
teamStats['xGA per game'] = teamStats['xGA'] / teamStats['MP']
teamStats['probability no goals allowed'] = poisson.pmf(0,teamStats['xGA per game'])
teamStats

,Squad,MP,W,D,L,GF,GA,GDiff,Pts,xG,xGA,xGDiff,xGDiff/90,Last 5,Attendance,Top Team Scorer,Goalkeeper,Notes,xGA per game,probability no goals allowed
Rk,,,,,,,,,,,,,,,,,,,,
1,Liverpool,31,28,2,1,70,21,49,86,59.7,30.9,28.8,0.93,W L W D W,53143,Mohamed Salah - 17,Alisson,NaN,0.996774,0.369068
2,Manchester City,31,20,3,8,77,33,44,63,72.2,29.4,42.8,1.38,W L W W L,54219,Sergio Agüero - 16,Ederson,NaN,0.948387,0.387365
3,Leicester City,31,16,7,8,59,29,30,55,48.6,37.4,11.2,0.36,L L W D D,32061,Jamie Vardy - 19,Kasper Schmeichel,NaN,1.206452,0.299257
4,Chelsea,31,16,6,9,55,41,14,54,54.7,30.0,24.7,0.80,W D W W W,40563,Tammy Abraham - 13,Kepa Arrizabalaga,NaN,0.967742,0.379940
5,Manchester Utd,31,13,10,8,48,31,17,49,48.3,29.9,18.4,0.59,W D W D W,72726,Marcus Rashford Anthony Martial - 14,David de Gea,NaN,0.964516,0.381168
6,Wolves,31,12,13,6,44,34,10,49,40.6,28.3,12.3,0.40,W W D W W,31360,Raúl Jiménez - 15,Rui Patrício,NaN,0.912903,0.401357
7,Tottenham,31,12,9,10,50,41,9,45,40.6,45.1,-4.5,-0.15,L L D D W,59384,Harry Kane - 12,Paulo Gazzaniga,NaN,1.454839,0.233438
8,Sheffield Utd,31,11,11,9,30,31,-1,44,34.7,39.2,-4.6,-0.15,D W D L L,30869,Lys Mousset John Fleck - 5,Dean Henderson,NaN,1.264516,0.282376
9,Arsenal,31,10,13,8,43,41,2,43,40.4,46.0,-5.6,-0.18,W W L L W,60279,Pierre-Emerick Aubameyang - 17,Bernd Leno,NaN,1.483871,0.226758


## xG-adjusted points

Next, we determine for each player their 'adjusted points'. To do this, we first subtract for each player all the points they have accumulated through goals, assists and clean sheets. Then, we add points for each player based on their expected goals, assists and clean sheets. This gives a much improved estimate of each player's true point generating capability. 

In [7]:
def incorporate_xG(indicator, ix):
    xG = playerStats.loc[indicator, 'xG'].values[0]  
    penalty_attempts = playerStats.loc[indicator,'PKatt'].values[0]
    if df.loc[ix, 'element_type']<=2:
        # 0.24 is the probability to miss a penalty, which incurs -2 points
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] - 6 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  5 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts
    elif df.loc[ix, 'element_type']==4:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'total_points'] -  4 * (df.loc[ix, 'goals_scored'] - xG) \
                                                                    - 2 * 0.24 * penalty_attempts

In [8]:
# always run 'team_xGA' AFTER 'incorporate_xG'
def team_xGA(indicator, ix):
    team = team_names[df.loc[ix, 'team']-1]
    clean_sheets = df.loc[ix, 'clean_sheets']
    probability_cleanSheet = teamStats.loc[teamStats['Squad']==team, 'probability no goals allowed'].values[0]
    if df.loc[ix, 'element_type']<=2:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    4 * (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)
    elif df.loc[ix, 'element_type']==3:
        df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  \
                    (df.loc[ix, 'clean_sheets'] - df.loc[ix, 'games played']*probability_cleanSheet)

A player who gives an assist that directly leads to a shot, is assigned the xG-value of the shot in xA (expected assists), i.e. xA is a measure of 'goal assists'. In FPL, however, the definition of an assist is somewhat more relaxed, e.g. goals resulting from a rebound of parried shot will award an assist to the player making the initial shot. For this reason, we calculate the total number of assists awarded in FPL and the total sum of xA of all players and get an estimate of the proportion of assists that xA covers in FPL. Then, we modify that proportion of players assists based on their xA.

In [9]:
# always run 'xA' AFTER 'incorporate_xG'
def xA(indicator, ix):
    xA = playerStats.loc[indicator, 'xA'].values[0]
    df.loc[ix, 'adjusted points'] =  df.loc[ix, 'adjusted points'] -  3 * (xA_proportion*df.loc[ix, 'assists'] - xA)

In [10]:
def calculateAdjustedPoints(indicator, index):
    incorporate_xG(indicator, index)
    team_xGA(indicator, index)
    xA(indicator, index)

In [11]:
team_names = np.sort(playerStats['Squad'].unique())
xA_proportion = playerStats['xA'].sum()/df['assists'].sum()
df['points_per_game'] = df['points_per_game'].astype(float)
df['games played'] = df['total_points']/df['points_per_game']
#df['games played'] = df['minutes']/90.0
xA_proportion

0.7740331491712709

## Main loop for assigning adjusted points

Below is the main loop where we calculate adjusted points for each player. Calculation of the adjusted points itself is straightforward, but there is some work required to match players in two different data sets. Comparing player names in both data sets gives unique matches in many cases, but some special cases need to be covered through individual solutions.

In [12]:
for ix in df[df['minutes']>0].index: 
    # player name in FPL data
    name = df.loc[ix, 'web_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u').replace('ö', 'o').\
                        replace('ä', 'a')
    # find FBREF data indexes where player name contains 'name'
    indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains(name)
    # if unique match is found, we can calculate 'adjusted points'
    if playerStats.loc[indicator].shape[0]==1:
        calculateAdjustedPoints(indicator, ix)
    # if no match is found
    elif playerStats.loc[indicator].shape[0]==0:
        # try matching based on first name, helps in some cases
        first_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '').replace('ü', 'u')\
                                                                    .replace('ö', 'o').replace('ä', 'a')
        first_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(first_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[first_name_indicator].shape[0]==1:
            calculateAdjustedPoints(first_name_indicator, ix)
        # here we deal with some special cases individually
        else:
            names = ['rodrigo','garcia','chicharito','elliott']
            if name == 'rodrigo':
                exceptional_case_indicator = \
                (playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodri')) & \
                (~playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '').str.contains('rodriguez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'chicharito':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('javierhernandez'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            elif name == 'elliott':
                exceptional_case_indicator = (playerStats['Player'].str.lower().str.replace(' ', '')\
                                              .str.replace('-', '').str.contains('elliot'))
                calculateAdjustedPoints(exceptional_case_indicator, ix)
            else:
                print(str(ix) + ': no player found.')
    # if more than one matches were found in the original comparison
    elif playerStats.loc[indicator].shape[0]>1:
        # try matching full name
        full_name = df.loc[ix, 'first_name'].lower().replace(' ', '').replace('-', '') \
            + df.loc[ix, 'second_name'].lower().replace(' ', '').replace('-', '')
        full_name_indicator = playerStats['Player'].str.lower().str.replace(' ', '').str.replace('-', '')\
                                                                            .str.contains(full_name)
        # if unique match is found, we can calculate 'adjusted points'
        if playerStats.loc[full_name_indicator].shape[0]==1:
            calculateAdjustedPoints(full_name_indicator, ix)
        # here we try to match players based both on their name and team
        else:
            team = team_names[df.loc[ix, 'team']-1]
            team_indicator = playerStats['Squad']==team
            if playerStats.loc[indicator & team_indicator].shape[0]==1:
                calculateAdjustedPoints(indicator & team_indicator, ix)
            # special case
            else:
                if name=='son':
                    exceptional_case_indicator = playerStats['Player'].str.lower().str.replace(' ', '')\
                                                .str.replace('-', '').str.contains('heung')
                    calculateAdjustedPoints(exceptional_case_indicator, ix)
                else:
                    print(str(ix) + ': non-unique name.')
    else:
        print(str(ix) + 'Player not found')

In [13]:
df['adjusted points per game'] = df['adjusted points'] / df['games played']

In [14]:
# give a sorted list showing the players with highest 'adjusted points per game'
df[['web_name', 'games played','total_points', 'points_per_game','adjusted points','adjusted points per game']]\
                                .sort_values(by='adjusted points per game', ascending=False)

,web_name,games played,total_points,points_per_game,adjusted points,adjusted points per game
id,,,,,,
191,Salah,26.986301,197,7.3,183.965086,6.816980
281,McGovern,2.000000,11,5.5,12.630879,6.315439
215,De Bruyne,29.130435,201,6.9,176.984230,6.075578
618,Fernandes,7.042254,50,7.1,42.357980,6.014833
192,Mané,27.910448,187,6.7,166.401960,5.961995
233,Rashford,24.166667,145,6.0,138.785304,5.742840
214,Sterling,26.000000,130,5.0,147.047299,5.655665
182,Alexander-Arnold,31.166667,187,6.0,171.301093,5.496292
166,Vardy,28.032787,171,6.1,147.767403,5.271235


In [15]:
# save data
filepath = '../data/data_week' + str(latest_gameweek) + str('.csv')
df.to_csv(filepath)

filepath = '../data/fbref/team_stats_week' + str(latest_gameweek) + '.csv'
teamStats.to_csv(filepath)

Below we check how well the total xG matches the total scored goals.

In [16]:
total_xG = playerStats['xG'].sum()
total_goals = df['goals_scored'].sum()
print('Total xG: ' + str(total_xG))
print('Total goals: ' + str(total_goals))
print('Total xG / total goals: ' + str(total_xG/total_goals))

Total xG: 840.1999999999999
Total goals: 805
Total xG / total goals: 1.043726708074534
